## A notebook to check the performance of OVIS on COCO novel class

In [ ]:
%matplotlib inline

## Part 1: The the class embeddings to match the saved embeddings

In [35]:
from pathlib import Path
import mmcv
import sys
import torch

In [21]:
BETRAY_PATH = "/jupyter-users-home/tan-2enguyen/betrayed-by-captions"

In [22]:
if BETRAY_PATH not in sys.path:
    sys.path.append(BETRAY_PATH)

In [41]:
known_class_file = '../datasets/unknown/known_65.txt'
class_emb_file = '../datasets/embeddings/coco_class_with_bert_emb.json'

In [57]:
file_client = mmcv.FileClient()

In [58]:
all_known_classes = file_client.get_text(known_class_file).split('\n')

In [59]:
import transformers
from open_set.models.utils.bert_embeddings import BertEmbeddings, BERT_MODEL_BY_EMBEDDING_TYPES

In [60]:
emb_type = 'bert'
tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL_BY_EMBEDDING_TYPES[emb_type])
bert_embeddings = BertEmbeddings(
    bert_model=transformers.AutoModel.from_pretrained(BERT_MODEL_BY_EMBEDDING_TYPES[emb_type]).eval(),
    normalize_word_embeddings=True  # Key to set this to TRUE
)

for param in bert_embeddings.parameters():
    param.requires_grad = False

/opt/conda/envs/betray/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bia

In [66]:
embs_by_name = {
    name: bert_embeddings.calculate_word_embeddings(
        torch.tensor(tokenizer.encode(name, add_special_tokens=False))
        ).squeeze() for name in all_known_classes
    }

In [96]:
gt_embs_by_name = {x['name']: torch.tensor(x['emb']) for x in mmcv.load(class_emb_file) if x['name'] in embs_by_name}

In [98]:
error_by_name = {k: torch.norm(v - gt_embs_by_name[k], p='fro') for k, v in embs_by_name.items()}

In [101]:
error_by_name

{'person': tensor(0.),
 'bicycle': tensor(0.),
 'car': tensor(0.),
 'motorcycle': tensor(1.7268e-06),
 'truck': tensor(3.3101e-06),
 'boat': tensor(1.6852e-06),
 'bench': tensor(0.),
 'bird': tensor(0.),
 'horse': tensor(0.),
 'sheep': tensor(0.),
 'zebra': tensor(0.),
 'giraffe': tensor(30.3267),
 'backpack': tensor(0.),
 'handbag': tensor(22.8224),
 'skis': tensor(23.0443),
 'kite': tensor(0.),
 'surfboard': tensor(21.2390),
 'bottle': tensor(0.),
 'spoon': tensor(3.8716e-06),
 'bowl': tensor(1.7392e-06),
 'banana': tensor(0.),
 'apple': tensor(0.),
 'orange': tensor(1.6809e-06),
 'broccoli': tensor(30.8248),
 'carrot': tensor(0.),
 'pizza': tensor(1.7574e-06),
 'donut': tensor(23.1388),
 'chair': tensor(0.),
 'bed': tensor(0.),
 'tv': tensor(0.),
 'laptop': tensor(0.),
 'remote': tensor(0.),
 'microwave': tensor(0.),
 'oven': tensor(1.6289e-06),
 'refrigerator': tensor(1.7703e-06),
 'book': tensor(0.),
 'clock': tensor(0.),
 'vase': tensor(0.),
 'toothbrush': tensor(21.1248),
 'trai

In [112]:
gt_embs_by_name['handbag'][:10]


tensor([-0.7196, -0.2812,  0.4077,  0.0171,  0.1302,  0.4881,  0.2265, -0.3112,
        -0.1379, -0.4031])

In [113]:
embs_by_name['elephant'][:10]

tensor([-0.1304,  0.6652,  0.6863, -0.4117,  0.4580,  0.1546, -0.6350,  0.7626,
         0.0577, -0.3102])

In [91]:
embs_by_name['handbag'][3]

tensor(1.0442)